In [1]:
import caffe
import numpy as np

In [2]:
IROSpath = '/home/jguerry/workspace/IROSmodels/'

target_net_proto = '/home/jguerry/workspace/IROSmodels/fcn32s_optim/deploy_optim.prototxt'
target_weights = '/home/jguerry/workspace/IROSmodels/fcn32s_optim/fcn32s_imagenet.caffemodel'

source_net_proto = '/home/jguerry/workspace/caffemodels/vgg16/VGG_ILSVRC_16_layers_deploy.prototxt'
source_weights = '/home/jguerry/workspace/caffemodels/vgg16/VGG_ILSVRC_16_layers.caffemodel'

In [3]:
def transfer(new_net, net, suffix=''):
    # from fcn.berkeleyvision.org
    for p in net.params:
        #print "Working on", p, ':'
        p_new = p + suffix
        if p_new not in new_net.params:
            print 'dropping', p
            continue
        for i in range(len(net.params[p])):
            if i > (len(new_net.params[p_new]) - 1):
                print 'dropping', p, i
                break
            if net.params[p][i].data.shape != new_net.params[p_new][i].data.shape:
                print 'coercing', p, i, 'from', net.params[p][i].data.shape, 'to', new_net.params[p_new][i].data.shape
            else:
                print 'copying', p, ' -> ', p_new, i
            new_net.params[p_new][i].data.flat = net.params[p][i].data.flat

In [ ]:
caffe_gpu = False
if not caffe_gpu:
    caffe.set_mode_cpu()
else:
    caffe.set_mode_gpu()
    caffe.set_device(0);

In [ ]:
target_net = caffe.Net(target_net_proto, caffe.TEST)
print "net loaded"

In [ ]:
target_net_blobs = [(k, v.data.shape) for k, v in target_net.blobs.items()]
print "blob exported"

In [ ]:
target_net_layers = [(k, v[0].data.shape, v[1].data.shape) for k, v in target_net.params.items() if 'deconv' not in k]
print "layers exported"

In [ ]:
source_net = caffe.Net(source_net_proto,source_weights, caffe.TEST)
#source_net_blobs = [(k, v.data.shape) for k, v in source_net.blobs.items()]
#source_net_layers = [(k, v[0].data.shape, v[1].data.shape) for k, v in source_net.params.items() if 'deconv' not in k]

In [ ]:
transfer(target_net,source_net)

In [ ]:
#### SAVING
print "Saving output model to %s " % target_weights
target_net.save(target_weights)
print "Saved."

In [ ]:
del target_net
del source_net